In [4]:
!pip install pyTelegramBotAPI
!pip install transformers
!pip install PyPDF2 nltk sumy
!pip install docx2txt
import telebot
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from transformers import pipeline
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:

# Set up Telegram Bot API token
bot_token = '6067346278:AAHjrvxKZfsa9QaqLfxANWsfe8ScD7y6YwY'
bot = telebot.TeleBot(bot_token)

@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "Welcome to the Summarization and Q&A Bot! Please send the file or text to be processed.\n"
                          "Supported file formats: PDF, DOCX, TXT\n"
                          "Send 'q' to quit.")

@bot.message_handler(content_types=['document'])
def process_document(message):
    chat_id = message.chat.id
    file_id = message.document.file_id
    file_info = bot.get_file(file_id)
    file_path = file_info.file_path

    file_extension = file_path.split('.')[-1].lower()
    if file_extension not in ['pdf', 'docx', 'txt']:
        bot.reply_to(message, "Invalid file format. Please send a PDF, DOCX, or TXT file.")
        return

    downloaded_file = bot.download_file(file_path)
    with open("temp." + file_extension, 'wb') as file:
        file.write(downloaded_file)

    text = extract_text_from_file(file_extension)
    summarize_and_answer(chat_id, text)

@bot.message_handler(func=lambda message: True)
def process_message(message):
    chat_id = message.chat.id
    user_input = message.text.strip()

    if user_input == 'q':
        bot.reply_to(message, "Exiting the bot...")
        bot.stop_polling()
    else:
        summarize_and_answer(chat_id, user_input)

def extract_text_from_file(file_extension):
    text = ""
    if file_extension == 'pdf':
        import PyPDF2
        with open("temp.pdf", 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text()
    elif file_extension == 'docx':
        import docx2txt
        text = docx2txt.process("temp.docx")
    elif file_extension == 'txt':
        with open("temp.txt", 'r') as file:
            text = file.read()

    return text

def summarize_and_answer(chat_id, text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, sentences_count=3)

    summary_text = "Summary:\n"
    for sentence in summary:
        summary_text += sentence.__str__() + "\n"

    bot.send_message(chat_id, summary_text)

    # Answering questions
    nlp = pipeline("question-answering")
    question = bot.send_message(chat_id, "Please ask a question based on the provided information or enter 'new' to process another file or text:")
    bot.register_next_step_handler(question, process_question, text)

def process_question(message, text):
    chat_id = message.chat.id
    user_input = message.text.strip()

    if user_input.lower() == 'new':
        bot.send_message(chat_id, "Please send the file or text to be processed.")
        return

    nlp = pipeline("question-answering")
    answer = nlp(question=user_input, context=text)

    if answer['score'] < 0.2:
        response = "I'm sorry, I couldn't find an answer to your question."
    else:
        response = f"Answer:\n{answer['answer']}"

    bot.send_message(chat_id, response)

    # Continue asking questions or process another file/text
    question = bot.send_message(chat_id, "Please ask another question or enter 'new' to process another file or text:")
    bot.register_next_step_handler(question, process_question, text)

# Start the bot
bot.polling()


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production 